# Лабораторная  №1 Вариант 13
##  Цель работы
 Изучить аналитеский (обратной матрицы) и численный (Брауна-Робинсон) методы нахождения смешнной стратегии в антогонистической игре двух лиц в нормальной форме

### Аналитический метод 
#### Метод обратной матрицы
Загрузим numpy и sys:

In [135]:
import sys
import numpy as np
import pandas as pd

np.set_printoptions(threshold=sys.maxsize)

Исходная матрица:

In [136]:
C = np.array([[9,10,13],
              [1,18,11],
              [17,4,0]])
C

array([[ 9, 10, 13],
       [ 1, 18, 11],
       [17,  4,  0]])

Посчитаем обратную ей:

In [137]:
C_inv = np.linalg.inv(C)
C_inv

array([[ 0.01794454, -0.02120718,  0.05057096],
       [-0.07626427,  0.09013051,  0.03507341],
       [ 0.12316476, -0.05464927, -0.06199021]])

In [138]:
u_C_inv= [np.array([[1,1,1]]) @ C_inv]
np.array(u_C_inv) 

array([[[0.06484502, 0.01427406, 0.02365416]]])

In [139]:
u_c_inv_u_t = u_C_inv @ np.array([1,1,1])
u_c_inv_u_t

array([[0.10277325]])

In [140]:
v = 1/u_c_inv_u_t
v

array([[9.73015873]])

Найдем оптимальную стратегию x:

In [141]:
x_opt = (np.array(u_C_inv)) * v
x_opt

array([[[0.63095238, 0.13888889, 0.23015873]]])

In [142]:
C_inv_u_t = C_inv @ [1,1,1]
C_inv_u_t

array([0.04730832, 0.04893964, 0.00652529])

Найдем оптимальную стратегию y:

In [143]:
y_opt = (np.array(C_inv_u_t)) * v
y_opt

array([[0.46031746, 0.47619048, 0.06349206]])

#### Ответ: x_opt=(0.63095238, 0.13888889, 0.23015873); y_opt=(0.46031746, 0.47619048, 0.06349206); v = 1.5

### Метод Брауна-Робинсон

Напишем метод Брауна-Робинсон

In [144]:
from math import inf
import random

C = np.array([[9,10,13],
              [1,18,11],
              [17,4,0]])
table=[]
INF=inf
cur_eps=INF # тк мы минимизируем ошибку

P = [1.0, 0.0, 0.0] #начальные стратегии
Q = [1.0, 0.0, 0.0] #начальные стратегии


v_niz=0
v_verh=0

result = []

def brown_robinson(result, eps):
    cur_strat_A = 0
    cur_strat_B = 0
    k = 0 # номер итерации
    cur_eps=INF
    
    dimension = len(C[0])
    
    #print(["k", "A", "B", "x[i]", "y[i]","v_verh/k", "v_niz/k", "eps" ])
    result.append([k,cur_strat_A+1, cur_strat_B+1, C[:,cur_strat_A], C[cur_strat_B],
                   np.amax(C[:,cur_strat_A])/(k+1), np.amin(C[cur_strat_B])/(k+1),
                   np.amax(C[:,cur_strat_A])/(k+1) - np.amin(C[cur_strat_B])/(k+1)]) # начальная итерация
    cur_strat_A, cur_strat_B = select_new_strats(C[:,cur_strat_A],
                                                 C[cur_strat_B]) #выбрали стратегии для след итерации
    
    k+=1
    
    while (cur_eps > eps):
        x_new, y_new = (np.add(C[:,cur_strat_B], result[k-1][3]),
                            np.add(C[cur_strat_A],result[k-1][4]))
        v_verh_new, v_niz_new = (np.amax(x_new)/(k+1), np.amin(y_new)/(k+1))
        
        cur_eps = calc_eps(result, v_verh_new, v_niz_new, k+1)

        new_appending = [k, cur_strat_A+1, cur_strat_B+1, x_new, y_new, v_verh_new, v_niz_new, cur_eps ]
        
        result.append(new_appending)
        
        cur_strat_A, cur_strat_B = select_new_strats(x_new,y_new)
                  
        k+=1
    
    strat_A_chosen, strat_B_chosen = calc_strat(result, dimension)
    
    print(np.array(result, dtype=object))  #РАСКОММЕНТИТЬ
    df = pd.DataFrame(result)
    df.to_csv('lab1.csv',index=False)
    return (result, (strat_A_chosen, strat_B_chosen))
    #return strat_A_chosen, strat_B_chosen
    #print(new_appending)    
     #   k+=1

def select_new_strats(arr_A, arr_B):
    ma, mi = (max(arr_A), min(arr_B))
    indexes_of_max = [i for i, j in enumerate(arr_A) if j == ma]
    indexes_of_min = [i for i, j in enumerate(arr_B) if j == mi]
    return (random.choice(indexes_of_max), random.choice(indexes_of_min))
    #return (np.argmax(arr_A), np.argmin(arr_B))    
    
def calc_eps(result, v_verh, v_niz, k):
    v_all_verh = np.append(np.array(result,dtype=object)[:,5], v_verh)
    v_all_niz = np.append(np.array(result, dtype=object)[:,6], v_niz)
    return (np.amin(v_all_verh)-np.amax(v_all_niz))

def calc_strat(result, dimension):
    N = len(result)
    strat_A_chosen, strat_B_chosen = {key+1: 0 for key in range(dimension)}, {key+1: 0 for key in range(dimension)}  
    
    for i in range(N):
        for j in strat_A_chosen:
            if result[i][1] == j:
                strat_A_chosen[j]=strat_A_chosen[j]+1
        for l in strat_B_chosen:
            if result[i][2] == l:
                strat_B_chosen[l]=strat_B_chosen[l]+1
    for m in range(dimension):
        strat_B_chosen[m+1]=strat_B_chosen[m+1]/N
        strat_A_chosen[m+1]=strat_A_chosen[m+1]/N
            
    print('1-ая прибл смеш стратегия: {strat_A} \n 2-ая прибл смеш стратегия: {strat_B} \n Число операций - {nums} \n\n'
          .format(strat_A=strat_A_chosen, strat_B=strat_B_chosen, nums=N))
    return (strat_A_chosen, strat_B_chosen)
    
table, (strat_A_BR, strat_B_BR) = brown_robinson(result, 0.1)

1-ая прибл смеш стратегия: {1: 0.6328257191201354, 2: 0.155668358714044, 3: 0.21150592216582065} 
 2-ая прибл смеш стратегия: {1: 0.4602368866328257, 2: 0.4450084602368866, 3: 0.09475465313028765} 
 Число операций - 591 


[[0 1 1 array([ 9,  1, 17]) array([ 9, 10, 13]) 17.0 9.0 8.0]
 [1 3 1 array([18,  2, 34]) array([26, 14, 13]) 17.0 6.5 8.0]
 [2 3 3 array([31, 13, 34]) array([43, 18, 13]) 11.333333333333334
  4.333333333333333 2.333333333333334]
 [3 3 3 array([44, 24, 34]) array([60, 22, 13]) 11.0 3.25 2.0]
 [4 1 3 array([57, 35, 34]) array([69, 32, 26]) 11.4 5.2 2.0]
 [5 1 3 array([70, 46, 34]) array([78, 42, 39]) 11.666666666666666 6.5
  2.0]
 [6 1 3 array([83, 57, 34]) array([87, 52, 52]) 11.857142857142858
  7.428571428571429 2.0]
 [7 1 3 array([96, 68, 34]) array([96, 62, 65]) 12.0 7.75 2.0]
 [8 1 2 array([106,  86,  38]) array([105,  72,  78]) 11.777777777777779
  8.0 2.0]
 [9 1 2 array([116, 104,  42]) array([114,  82,  91]) 11.6 8.2 2.0]
 [10 1 2 array([126, 122,  46]) array

#### График eps от n

In [145]:
import plotly.express as px

fig = px.scatter(x=[i for i in range(len(table))], y=[i[7] for i in table])
fig.show()

### Расчитаем Δ:

In [146]:
x_BR, y_BR = list(strat_A_BR.values()), list(strat_B_BR.values()) # dict->list

delt_x, delt_y = (abs(np.subtract(x_opt, x_BR)), abs(np.subtract(y_opt, y_BR)))
print("Погрешность для опт смеш страт А: {},\n Погрешность для опт смеш страт B: {}".format(delt_x, delt_y))


Погрешность для опт смеш страт А: [[[0.00187334 0.01677947 0.01865281]]],
 Погрешность для опт смеш страт B: [[8.05736846e-05 3.11820160e-02 3.12625896e-02]]
